In [5]:

import numpy as np
import os
from pymoo.core.problem import Problem
from utils import simulation, init_env, run_pymoo_algorithm, run_pymoo_algorithm
from fitness_functions import original_fitness, individual_gain


from evoman.environment import Environment
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch
from pymoo.optimize import minimize
from pymoo.problems.single import Himmelblau

In [2]:
class objectives(Problem):
    enemies: list[int]
    env: Environment

    def __init__(self, env: Environment, n_genes: int, enemies: list[int], n_objectives):
        self.env = env
        self.enemies = enemies
        super().__init__(n_var=n_genes, n_obj=n_objectives, xl=-1, xu=1, type_var=float)

    def _evaluate(self, x: list[np.array], out, *args, **kwargs):
        """Evaluate the fitness of each individual in the population
        We can turn on elementwise_evaluation to evaluate each individual in the population separately
        https://pymoo.org/problems/parallelization.html#Custom-Parallelization

        x - list of individuals in the population
        out - dictionary with the fitness outputs

        # when we have multiple enemies we need to average the fitness somehow
        # Ideas: mean of the fitness will show how agent performs with both enemies
        #        max will show how agent performs with the worst enemy (this does not reflect the performance with
        #        another enemy)
        #        weighted average is another option, but then we have another problem of how to weight the enemies
        """
        if POP_SIZE != len(x):
            print(f"WARNING: POP_SIZE != len(x) in evaluation step (this happens sometimes do not see why)\n"
                  f"pop size: {POP_SIZE}; len x:{len(x)}")
        # Initialize
        dict_enemies = {}
        # Get fitness for each enemy
        for enemy in self.enemies:
            self.env.update_parameter('enemies', [enemy])

            dict_enemies[enemy] = []
            for individual_id in range(len(x)):
                dict_enemies[enemy].append(
                    simulation(self.env, x[individual_id], inverted_fitness=True, fitness_function=individual_gain))

        # Return fitness outputs for enemies
        objectives_fitness = {
            "objective_hard": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [1, 6]]) for ind_id in
                               range(len(x))],
            "objective_medium": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [2, 5, 8]]) for ind_id in
                                 range(len(x))],
            "objective_easy": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [3, 4, 7]]) for ind_id in
                               range(len(x))],
        }


In [3]:
solutions = []
for file in os.listdir('solutions_beats_5_enemies'):
    if file.startswith('pymoo'):
        new_solution = []
        with open(f'solutions_beats_5_enemies/{file}') as f:
            solutions.append(f.read().splitlines())
            
solutions = np.array(solutions, dtype=float)

In [4]:
ENEMIES = [2, 3, 6]
env, n_genes = init_env('local_search_test', ENEMIES, 10)
problem = objectives(
        env=env,
        n_genes=n_genes,
        enemies=ENEMIES,
        n_objectives=1)

In [16]:
POP_SIZE = solutions.shape[0]

algorithm = PatternSearch(x0=solutions)
algorithm.setup(problem, termination=('n_gen', 5), verbose=False)
algorithm = run_pymoo_algorithm(algorithm, problem)
res = algorithm.result()
res

SystemExit: 0

c:\Users\beuze\anaconda3\envs\ec110\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
%tb

SystemExit: 0